In [1]:
from sqlalchemy import create_engine, inspect, text
import pandas as pd

engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

In [2]:
def run_sql_query(query=None):
    if not query:
        query=input()
    with engine.connect() as conn:
        rs = conn.execute(text(query))
        df = pd.DataFrame(rs.fetchall())
    return df

In [3]:
f = run_sql_query("SELECT * FROM Facilities")
b = run_sql_query("SELECT * FROM Bookings")
m = run_sql_query("SELECT * FROM Members")

In [4]:
def disp_cols(title,df):
    print(f"{title.upper()}")
    for col in df.columns:
        print(f"\t{col}")

In [5]:
disp_cols('facilities',f)

FACILITIES
	facid
	name
	membercost
	guestcost
	initialoutlay
	monthlymaintenance


In [6]:
disp_cols('bookings',b)

BOOKINGS
	bookid
	facid
	memid
	starttime
	slots


In [7]:
disp_cols('members',m)

MEMBERS
	memid
	surname
	firstname
	address
	zipcode
	telephone
	recommendedby
	joindate


In [8]:
[ col for col in f.columns if col in b.columns ]

['facid']

i.e., can join **```Facilities```** with **```Bookings```** on *```facid```* column

In [9]:
[ col for col in f.columns if col in m.columns ]

[]

So, no common column to join **```Facilities```** with **```Members```** (will need to use **```Bookings```** as intermediary)

In [10]:
[ col for col in b.columns if col in m.columns ]

['memid']

i.e., can join **```Bookings```** with **```Members```** on *```memid```* column.

### Question 10
/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

In [11]:
query="""
SELECT facility, SUM(cost) AS total_revenue

FROM (
    SELECT 
        f.name AS facility,
        (CASE WHEN m.memid = 0 
        THEN slots * guestcost
        ELSE slots * membercost END) AS cost
        
    FROM Members AS m
    INNER JOIN Bookings AS b
        ON m.memid = b.memid
    
    INNER JOIN Facilities as f
        ON f.facid = b.facid
) AS subq

GROUP BY facility
HAVING SUM(cost) < 1000
ORDER BY total_revenue

"""
run_sql_query(query)

,facility,total_revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


### Question 11
/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

In [12]:
query="""

SELECT 
    member_name,
    recommender_name

FROM(
    SELECT 
        recommendedby AS id,
        CONCAT_WS(', ',surname,firstname) AS member_name
    FROM Members
    WHERE recommendedby != ''
) AS main

INNER JOIN(
    SELECT
        memid AS id,
        CONCAT_WS(', ',surname,firstname) AS recommender_name
    FROM Members
    WHERE recommendedby != ''
) AS recs

ON main.id = recs.id
ORDER BY member_name, recommender_name ASC
        
"""
run_sql_query(query)

,member_name,recommender_name
0,"Bader, Florence","Stibbons, Ponder"
1,"Baker, Anne","Stibbons, Ponder"
2,"Coplin, Joan","Baker, Timothy"
3,"Dare, Nancy","Joplette, Janice"
4,"Genting, Matthew","Butters, Gerald"
5,"Hunt, John","Purview, Millicent"
6,"Jones, David","Joplette, Janice"
7,"Jones, Douglas","Jones, David"
8,"Rumney, Henrietta","Genting, Matthew"
9,"Sarwin, Ramnaresh","Bader, Florence"


### Question 12
/* Q12: Find the facilities with their usage by member, but not guests */

In [13]:
query="""
SELECT
    f.facility,
    COUNT(m.memid) AS member_bookings

FROM (SELECT memid FROM Members WHERE memid != 0) AS m

INNER JOIN (SELECT bookid,facid,memid FROM Bookings) AS b
    ON m.memid = b.memid

INNER JOIN (SELECT facid, name AS facility FROM Facilities) AS f
    ON f.facid = b.facid

GROUP BY f.facility
"""
run_sql_query(query)

,facility,member_bookings
0,Badminton Court,344
1,Massage Room 1,421
2,Massage Room 2,27
3,Pool Table,783
4,Snooker Table,421
5,Squash Court,195
6,Table Tennis,385
7,Tennis Court 1,308
8,Tennis Court 2,276


#### Question 13
/* Q13: Find the facilities usage by month, but not guests */

In [14]:
query="""
SELECT
    facility,
    STRFTIME('%m', b.starttime) AS month,
    COUNT(m.memid) AS member_bookings

FROM (SELECT memid FROM Members WHERE memid != 0) AS m

INNER JOIN (SELECT bookid,facid,memid, starttime FROM Bookings) AS b
    ON m.memid = b.memid

INNER JOIN (SELECT facid, name AS facility FROM Facilities) AS f
    ON f.facid = b.facid

GROUP BY facility,STRFTIME('%m', b.starttime)
"""
run_sql_query(query)

,facility,month,member_bookings
0,Badminton Court,07,51
1,Badminton Court,08,132
2,Badminton Court,09,161
3,Massage Room 1,07,77
4,Massage Room 1,08,153
5,Massage Room 1,09,191
6,Massage Room 2,07,4
7,Massage Room 2,08,9
8,Massage Room 2,09,14
9,Pool Table,07,103
